In [ ]:
import os

import numpy as np
import torch
import torch.nn as nn

from instruction_following.settings import ModelSettings

General settings

In [ ]:
# Training data
block_size = ModelSettings.max_context_length
batch_size = 32

# Learning
max_iters = 600000  # total number of training iterations
learning_rate = 6e-4
min_lr = 6e-5
lr_decay_steps = max_iters  # should be ~= max_iters per Chinchilla
warmup_steps = 2000

Hardware settings

In [ ]:
torch.backends.cuda.matmul.allow_tf32 = True  # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True  # allow tf32 on cudnn
device = "cuda" if torch.cuda.is_available() else "cpu"
device_type = 'cuda' if 'cuda' in device else 'cpu'

Training data stream

In [ ]:
data_dir = "tokenized_data"


def get_batch(split):
    # We recreate np.memmap every batch to avoid a memory leak, as per
    # https://stackoverflow.com/questions/45132940/numpy-memmap-memory-usage-want-to-iterate-once/61472122#61472122
    if split == 'train':
        data = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint16, mode='r')
    else:
        data = np.memmap(os.path.join(data_dir, 'test.bin'), dtype=np.uint16, mode='r')
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([torch.from_numpy((data[i:i + block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i + 1:i + 1 + block_size]).astype(np.int64)) for i in ix])
    if device == 'cuda':
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    return x, y

Scaler for FP16

In [ ]:
scaler = torch.cuda.amp.GradScaler()

Model settings

In [ ]:
from model import ChatModel
from settings import ModelSettings

model = ChatModel(
    vocabulary_size=ModelSettings.vocabulary_size,
    embedding_size=ModelSettings.embedding_size,
    embedding_dropout=0.0,
    attention_dropout=0.0,
    max_context_length=ModelSettings.max_context_length,
    ff_size_multiplier=ModelSettings.ff_size_multiplier,
    ff_dropout=0.0,
    transformer_blocks=ModelSettings.transformer_blocks,
    attention_heads=ModelSettings.attention_heads
)

Optimizer

In [ ]:
from optimizer import get_optim_groups

optim_groups = get_optim_groups(model)

optimizer = torch.optim.AdamW(
    optim_groups,
    lr=3e-4,
    betas=(0.9, 0.95),
    eps=1e-8
)


Checkpointer

In [ ]:
checkpoint_dir = "checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)


def save_checkpoint(step, model, optimizer, scaler, train_loss, val_loss):
    checkpoint = {
        "step": step,
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict(),
        "scaler": scaler.state_dict() if scaler else None,
        "train_loss": train_loss,
        "val_loss": val_loss
    }

    torch.save(checkpoint, f"{checkpoint_dir}/checkpoint_{step}.pt")

Load training state

In [ ]:
def load_checkpoint():
    checkpoint = torch.load(f"{checkpoint_dir}/checkpoint_{step}.pt")
    if step != checkpoint["step"]:
        raise Exception(f" {step}")
    model.load_state_dict(checkpoint["model"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    scaler.load_state_dict(checkpoint["scaler"])


Learning scheduler

In [ ]:
import math


def get_lr(step):
    # 1) linear warmup for warmup_steps steps
    if step < warmup_steps:
        return learning_rate * (step + 1) / (warmup_steps + 1)
    # 2) if it > lr_decay_steps, return min learning rate
    if step > lr_decay_steps:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (step - warmup_steps) / (lr_decay_steps - warmup_steps)
    assert 0 <= decay_ratio <= 1
    coefficient = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))  # coefficient ranges 0..1
    return min_lr + coefficient * (learning_rate - min_lr)

Training settings

In [ ]:
patience = 5  # number of evaluations to wait
min_delta = 0.02  # minimum improvement
grad_clip = 1.0

Training loop

In [ ]:
best_loss = float("inf")
patience_counter = 0

for step in range(50000):
    optimizer.zero_grad()

    xb, yb = get_batch("train")

    with torch.amp.autocast(dtype=torch.float16, device_type=device_type):
        logits, loss = model(xb, yb)

    scaler.scale(loss).backward()
    scaler.unscale_(optimizer)

    # Prevent too large gradients with clipping
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    scaler.step(optimizer)
    scaler.update()

    optimizer.step()

    if step % 10 == 0:
        loss_num = loss.item()
        perplexity = math.exp(loss_num)
        print(f"step {step}, loss {loss_num:.4f}, perplexity {perplexity:.2f}")
        if best_loss - loss_num > min_delta:
            best_loss = loss_num
            patience_counter = 0
            save_checkpoint(
                step=step,
                model=model,
                optimizer=optimizer,
                scaler=scaler,
                train_loss=loss,
                val_loss=0
            )
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping triggered")
            break

Test the model

In [ ]:
from tokenizers.tokenizers import Tokenizer

tokenizer = Tokenizer.from_file("tokenizer.json")


@torch.no_grad()
def generate(model, start, max_new_tokens=50):
    model.eval()
    idx = torch.tensor([tokenizer.encode(start).ids], device=device, dtype=torch.long)

    for _ in range(max_new_tokens):
        idx_cond = idx[:, -ModelSettings.max_context_length:]
        logits = model(idx_cond)
        logits = logits[:, -1, :]
        probs = nn.functional.softmax(logits, dim=-1)
        next_id = torch.multinomial(probs, num_samples=1)
        idx = torch.cat([idx, next_id], dim=1)

    return tokenizer.decode(idx[0].tolist())


start_token_id = get_batch("test")[0][0][0].item()
start_text = tokenizer.decode([start_token_id])
print(generate(model, start_text))

In [ ]:
from google.colab import drive

drive.mount('/content/drive')